In [1]:
#Teste Fumio"
#Validando commit, push, branch e codspace"
#Este comentário está sendo inscrito no VSCODE do Codspace.
    #Instalei o Pyspark no Codspace.

# Importa o Spark e outras bibliotecas necessárias
import pyspark # O módulo principal
from pyspark.sql import SparkSession
from pyspark.sql import functions as F # O alias 'F' MAIÚSCULO é usado para todas as funções (F.col, F.when, etc.)
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType

# Cria ou obtém uma SparkSession
# Master: 'local[*]' indica que o Spark deve usar todos os núcleos disponíveis na máquina
# appName: Nome da aplicação (importante para monitoramento)
spark = (SparkSession.builder
    .master("local[*]")
    .appName("AulaPraticaPySpark_DataQuality")
    .config("spark.executor.memory", "4g") # Opcional: Configurações de memória
    .config("spark.driver.memory", "4g")  # Opcional: Configurações de memória
    .getOrCreate()
)

# Imprime o status da sessão para confirmação
print("SparkSession inicializada com sucesso!")
print(f"Versão do Spark: {spark.version}")
print(f"Aplicação: {spark.sparkContext.appName}")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/02 16:06:13 WARN Utils: Your hostname, codespaces-844813, resolves to a loopback address: 127.0.0.1; using 10.0.10.206 instead (on interface eth0)
25/11/02 16:06:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/02 16:06:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


SparkSession inicializada com sucesso!
Versão do Spark: 4.0.1
Aplicação: AulaPraticaPySpark_DataQuality
